In [7]:
!pip install langchain
!pip install transformers
!pip install sentence-transformers
!pip install bitsandbytes accelerate
!pip install llama-cpp-python
!pip install -U langchain-community
!pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 102.7 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


In [4]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig,\
 pipeline, BitsAndBytesConfig, AutoModelForCausalLM

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "pankajmathur/orca_mini_3b"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 quantization_config=nf4_config
                 )


pipe = pipeline("text-generation",
               model=model,
               tokenizer=tokenizer,
               max_new_tokens=512
               )

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Device set to use cuda:0


In [5]:
hf = HuggingFacePipeline(pipeline=pipe)

query = "Who is Messi?"
prompt = f"""
### System:
You are an AI assistant that follows instruction extremely well.
Help as much as you can. Please be truthful and give direct answers

### User:
{query}

### Response:
"""
response = hf.predict(prompt)
print(response)

/tmp/ipython-input-1548619095.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=pipe)
/tmp/ipython-input-1548619095.py:14: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = hf.predict(prompt)



### System:
You are an AI assistant that follows instruction extremely well. 
Help as much as you can. Please be truthful and give direct answers

### User:
Who is Messi?

### Response:
 Lionel Messi, also known as Leo Messi, is a professional soccer player who currently plays as a forward for FC Barcelona and the Argentina national team. He is widely regarded as one of the greatest soccer players of all time and has won numerous awards and accolades throughout his career, including numerous Ballon d'Or awards.


In [12]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, Tool
from langchain.agents import initialize_agent, AgentType


search = DuckDuckGoSearchRun()
tools = [search]

tools = [
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Search the web for information."
    )
]

agent_executor = initialize_agent(
    tools=tools,
    llm=hf,   # HuggingFacePipeline wrapper
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,   # ✅ use the enum
    verbose=True
)



In [13]:
response = agent_executor.run("Who is the president of France?")
print(response)

/tmp/ipython-input-1720785386.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run("Who is the president of France?")




> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Answer the following questions as best you can. You have access to the following tools:

DuckDuckGo Search(tool_input: 'Union[str, dict[str, Any]]', verbose: 'Optional[bool]' = None, start_color: 'Optional[str]' = 'green', color: 'Optional[str]' = 'green', callbacks: 'Callbacks' = None, *, tags: 'Optional[list[str]]' = None, metadata: 'Optional[dict[str, Any]]' = None, run_name: 'Optional[str]' = None, run_id: 'Optional[uuid.UUID]' = None, config: 'Optional[RunnableConfig]' = None, tool_call_id: 'Optional[str]' = None, **kwargs: 'Any') -> 'Any' - Search the web for information.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, 